# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [4]:
! pip install -q schedule pytest

In [16]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import datetime
import time
import requests
import schedule
import json
import os
import pytest  # noqa: F401
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [ ]:
def get_book_data(book_url: str) -> dict:
    """
    Функция для получения данных о книге по url

    Функция должна возвращать определенную информацию по книге:
    - Название
    - Цена
    - Рейтинг
    - Количество в наличии
    - Описание
    - Дополнительные характеристики

    Функция должна возвращать словарь в формате словаря:
    {
        'name': 'Название книги',
        'price euro': 'Цена',
        'rating': 'Рейтинг',
        'stock': 'Количество в наличии',
        'description': 'Описание',
        'Product Information': 'Дополнительные характеристики'  --Данные список будет выводится по всем возможным дополнительным параметрам
     }

     аргументы:
     book_url (str): ссылка на определенный сайт с книгой в формате str

     return:
     dict: словарь с данными о книге

    >>>book_url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
    >>>get_book_data(book_url)

        {'name': 'A Light in the Attic',
        'price euro': 51.77,
        'rating': 3.0,
        'stock': 22,
        'Decription': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby, I think someone down here'sGot it in for you. Shel, you never sounded so good. ...more",
        'UPC': 'a897fe39b1053632',
        'Product Type': 'Books',
        'Tax': 0.0,
        'Number of reviews': '0'}
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    # Создаем словарь с ключами
    book_dict = {}
    # Перевод рейтинга в числа
    rating_book = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
    # список дополнительных данных, которые мы не выгружаем, так как уже подобные есть
    book_exception_description = {
        "Price (excl. tax)",
        "Price (incl. tax)",
        "Availability",
    }

    # Получаем данные о книге. Проверяем подключение
    try:
        timeout = (7, 11)
        req_book = requests.get(book_url, timeout=timeout)
        req_book.raise_for_status()
    except req_book.HTTPError as err:  # type: ignore
        return print(f"Возникла ошибка: {err}")  # type: ignore # надо ли вообще?

    # Имеем HTML формат. Так как мы парсим опредлеленный сайт, то мы не проверяем тип данных
    # print(req_book.headers["Content-Type"])

    # Проверяем кодировку
    # current_encoding = req_book.encoding
    # print(f"Текущая кодировка: {current_encoding}")
    req_book.encoding = "utf-8"

    # Парсим данные по html
    req_book_html = BeautifulSoup(req_book.text, "html.parser")

    # 1. Заранее определяем, что h1 - название книги. Ищем и вставляем название в словарь
    book_name = req_book_html.find("h1").text.strip()  # type: ignore
    book_dict.update({"name": str(book_name)})

    # 2. Заранее определяем, что p - цена книги. Ищем и вставляем название в словарь
    book_price = req_book_html.find("p", class_="price_color").text.strip()[1:]  # type: ignore
    book_dict.update({"price euro": float(book_price)})

    # 3. Рейтинг (ищем элемент с классом star-rating и извлекаем подкласс)
    rating_element = req_book_html.find("p", class_="star-rating")
    rating_classes = list(rating_element.get("class", []))  # type: ignore
    book_dict.update({"rating": float(rating_book.get(rating_classes[1]))})  # type: ignore

    # 4. Остатки (ищем элемент с классом p - instock availability извлекаем позиции чисел)
    stock_element_start = (
        req_book_html.find("p", class_="instock availability").text.strip().find("(")  # type: ignore
        + 1
    )
    stock_element_end = (
        req_book_html.find("p", class_="instock availability")
        .text.strip()  # type: ignore
        .find(" available")
    )
    # print(req_book_html.find("p", class_="instock availability").text.strip()[stock_element_start:stock_element_end])
    book_dict.update(
        {
            "stock": int(
                req_book_html.find("p", class_="instock availability").text.strip()[  # type: ignore
                    stock_element_start:stock_element_end
                ]
            )
        }
    )

    # 5. Описание (ищем элемент с классом star-rating и извлекаем подкласс)
    try:
        description_element = (
            req_book_html.find("div", id="product_description")
            .find_next("p")  # type: ignore
            .text.strip()  # type: ignore
        )
        book_dict.update({"Decription": str(description_element)})
    except:  # noqa: E722
        book_dict.update({"Decription": "No description"})

    # 6. Product Information. По всем строкам table/tr ищем по th - key, по td - value.
    product_info_element_key = (
        req_book_html.find("h2", string="Product Information")  # type: ignore
        .find_next("table")
        .find_all("tr")
    )

    for row in product_info_element_key:
        try:
            key = row.find("th").text.strip()
            if key in book_exception_description:
                continue
            value = row.find("td").text.strip()
            #
            book_dict[key] = value if key != "Tax" else float(value[1:])
        except:  # noqa: E722
            continue
        # pass

    return book_dict
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [9]:
# Используйте для самопроверки
book_url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
get_book_data(book_url)

{'name': 'A Light in the Attic',
 'price euro': 51.77,
 'rating': 3.0,
 'stock': 22,
 'Decription': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up t

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [17]:
def scrape_books(is_save: bool, pages_url: str, directory_save: str):
    """
    Функция для получения данных о книгах на определенном сайте

    Функция должна искать ссылки на все страницы книг
    далее при помощи функции get_book_data собрать данные о каждой книге и вернуть их.
    Если сайт оканчивается (ошибка 404), то функция останавливается.
    В случае если is_save == True, сохраните данные в JSON файл.

    В итоге возвращает список книг с такими данными:
    - Название
    - Цена
    - Рейтинг
    - Количество в наличии
    - Описание
    - Дополнительные характеристики

    Аргументы:
    is_save (bool): если True, то создает файл с нуля и сохраняет в него данные.
    В противном случае - ничего не делает.
    Сохраняет файл artifacts/books_data.txt в формате JSON

    pages_url (str): ссылка на сайт с книгами. Выглядит как шаблон с переменной N - номер страницы.

    Возвращает:
    list[dict]: список словарей с данными о книгах.

    Пример:
        books = scrape_books(
            is_save=True,
            pages_url="http://books.toscrape.com/catalogue/page-{N}.html"
        )
        print(len(books))

        >>>1000
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    # Стартуем с первой страницы. Готовим первую ссылку
    pages_site = 1
    url_site = pages_url.format(N=pages_site)
    book_list = []
    # Будем парсить сайты, пока на наткнемся на 404 ошибку (51 стр.)
    while True:
        timeout = (7, 11)
        req_pages = requests.get(url_site, timeout=timeout)
        # проверка на ошибку
        if req_pages.status_code == 404:
            break

        # Меняем кодировку
        req_pages.encoding = "utf-8"
        # Парсим данные по html
        req_pages_html = BeautifulSoup(req_pages.text, "html.parser")
        # Определяем секцию, в которой расположены книги
        req_pages_html_find = req_pages_html.find("ol", class_="row").find_all(  # type: ignore
            "article", class_="product_pod"
        )
        book_i = 1
        # Для каждой книги определяем ссылку.
        # Далее вытаскиваем данные по каждой книге по ссылке. Сохраняем в список
        for book_pages in req_pages_html_find:
            book_url = book_pages.find("a").get("href")  # type: ignore
            percent_time = ((pages_site - 1) * 20 + book_i) / (50 * 20) * 100
            print(f"{percent_time:.2f}%", end="\r", flush=True)
            book_list.append(
                get_book_data(f"http://books.toscrape.com/catalogue/{book_url}")
            )
            book_i += 1
        pages_site += 1
        url_site = pages_url.format(N=pages_site)

    # Сохраняем в файл в формате json, если указан True
    if is_save:
        try:
            with open(f"{directory_save}/books_data.txt", "w") as f:
                json.dump(book_list, f, indent=4)
        except FileNotFoundError:
            os.makedirs(directory_save)
            with open(f"{directory_save}/books_data.txt", "w") as f:
                json.dump(book_list, f, indent=4)

    return book_list


In [ ]:
# Проверка работоспособности функции
res = scrape_books(
    is_save=True,
    pages_url="http://books.toscrape.com/catalogue/page-{N}.html",
    directory_save="artifacts",
)  # Допишите ваши аргументы
# res = scrape_books(is_save = False)  # Допишите ваши аргументы
print(type(res), len(res))  # и проверки

<class 'list'> 1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
schedule.every().day.at("17:25").do(
    scrape_books, True, "http://books.toscrape.com/catalogue/page-{N}.html", "artifacts"
)
hours_sleep = 1
while True:
    schedule.run_pending()
    time.sleep(60)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

# Работает????

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [5]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest tests/test_scraper.py

============================= test session starts =============================
platform win32 -- Python 3.13.7, pytest-8.4.2, pluggy-1.6.0
rootdir: d:\03_IT\01_Scripts\01_DZ3_Python
plugins: mock-3.15.1
collected 0 items / 1 error

=================================== ERRORS ====================================
___________________ ERROR collecting tests/test_scraper.py ____________________
tests\test_scraper.py:66: in <module>
    result = scrape_books(is_save = True, pages_url="http://books.toscrape.com/catalogue/page-{N}.html", directory_save="artifacts")
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
scraper.py:221: in scrape_books
    get_book_data(f"http://books.toscrape.com/catalogue/{book_url}")
C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\unittest\mock.py:1169: in __call__
    return self._mock_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
C:\Users\user\AppData

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```